# Experiment 1: Hyperparameters

In [ ]:
from data_classes.TextLightningDataModule import TextLightningDataModule
from models.ClassifierSystem import LightningClassifier
from data_classes.pretrained_embeddings import get_pretrained_embeddings
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pandas as pd

In [ ]:
# Data and model settings
dataset = "IMDB"
num_class = 2
embedding = "Glove"
max_vectors = 10000
dim = 300


# Training settings
max_epochs=12
patience=4
lr=1e-3
batch_size=32
num_workers=0
advanced_metrics=True

# Log file:
log_file = "exp1"
log_file_csv = "exp1_csv"

In [ ]:
vocab, vectors = get_pretrained_embeddings(embedding =embedding, max_vectors = max_vectors, dim = dim)

In [ ]:
imdb_data = TextLightningDataModule(vocab, dataset=dataset, batch_size=batch_size, num_workers=num_workers)

In [ ]:
for model_type in ["RNN", "LSTM", "GRU"]:
    for output_layer_type in ["linear", "MLP"]:
        for num_layers in [1, 2]:
            for hidden_size in [100, 300]:
                name = "exp1-" + model_type + "-" + output_layer_type + "-" + str(num_layers) + "-" + str(hidden_size) 
                logger_tensor = TensorBoardLogger(log_file, name=name)
                logger_csv = CSVLogger(log_file_csv, name=name)
                trainer = Trainer(max_epochs=max_epochs, gpus=1, auto_select_gpus=True, callbacks=[EarlyStopping(monitor="Val Loss", patience=patience)], logger=[logger_tensor, logger_csv])
                classifier = LightningClassifier(num_class=num_class, vocab=vocab, vectors=vectors, embedding_size=dim, learning_rate=lr, num_layers=num_layers, hidden_size=hidden_size, model_type=model_type, attention_type=None, output_layer_type=output_layer_type, advanced_metrics=advanced_metrics)
                trainer.fit(classifier, imdb_data)
                trainer.test(ckpt_path="best")

In [ ]:
# Combine results into a single csv
df_list = []
for model_type in ["RNN", "LSTM", "GRU"]:
    for output_layer_type in ["linear", "MLP"]:
        for num_layers in [1, 2]:
            for hidden_size in [100, 300]:
                name = "exp1-" + model_type + "-" + output_layer_type + "-" + str(num_layers) + "-" + str(hidden_size) 
                filename = "exp1_csv/"+name+"/version_0/metrics.csv"
                df = pd.read_csv(filename)
                df["Model Type"] = model_type
                df["Output Layer Type"] = output_layer_type
                df["Number of Layers"] = num_layers
                df["Hidden Size"] = hidden_size
                df_list.append(df)
global_df = pd.concat(df_list)
global_df.to_csv("results/exp1_all.csv")

In [ ]:
# Parameter counts
# param_count = {}
# pytorch_total_params = sum(p.numel() for p in classifier.parameters() if p.requires_grad)
# param_count[name] = pytorch_total_params

# param_count
# {'exp1-RNN-linear-1-100': 40402,
#  'exp1-RNN-linear-1-300': 181202,
#  'exp1-RNN-linear-2-100': 60602,
#  'exp1-RNN-linear-2-300': 361802,
#  'exp1-RNN-MLP-1-100': 80702,
#  'exp1-RNN-MLP-1-300': 542102,
#  'exp1-RNN-MLP-2-100': 100902,
#  'exp1-RNN-MLP-2-300': 722702,
#  'exp1-LSTM-linear-1-100': 161002,
#  'exp1-LSTM-linear-1-300': 723002,
#  'exp1-LSTM-linear-2-100': 241802,
#  'exp1-LSTM-linear-2-300': 1445402,
#  'exp1-LSTM-MLP-1-100': 201302,
#  'exp1-LSTM-MLP-1-300': 1083902,
#  'exp1-LSTM-MLP-2-100': 282102,
#  'exp1-LSTM-MLP-2-300': 1806302,
#  'exp1-GRU-linear-1-100': 120802,
#  'exp1-GRU-linear-1-300': 542402,
#  'exp1-GRU-linear-2-100': 181402,
#  'exp1-GRU-linear-2-300': 1084202,
#  'exp1-GRU-MLP-1-100': 161102,
#  'exp1-GRU-MLP-1-300': 903302,
#  'exp1-GRU-MLP-2-100': 221702,
#  'exp1-GRU-MLP-2-300': 1445102}